In [ ]:
#!jupyter nbconvert --to=python NPP_TotalGlobal_python3.ipynb

In [ ]:
class NPP_TotalSO:
    '''
    class NPP_TotalSO(runname,resultpath,savepath,meshpath,first_year,last_year,
                 mapproj='pc',savefig=False,layerwise=False)
    '''
    def __init__(self,runname,resultpath,savepath,meshpath,first_year,last_year,
                 mapproj='pc',
                 savefig=False,
                 layerwise=False,depth_array=[]):

        self.runname = runname
        self.resultpath = resultpath
        self.savepath = savepath
        self.meshpath = meshpath
        self.fyear = first_year
        self.lyear = last_year
        self.mapproj = mapproj
        self.savefig = savefig
        self.layerwise = layerwise
        self.depth_array = depth_array

        import matplotlib.pyplot as plt
        import numpy as np
        from netCDF4 import Dataset
        from scipy.interpolate import griddata
        import skill_metrics as sm
        import cartopy.crs as ccrs
        #import pickle

        import pyfesom2 as pf


        # load FESOM mesh -------------------------------------------------------------------------------------
        mesh       = pf.load_mesh(meshpath)
        years = range(self.fyear, self.lyear+1)
        
        ## Souther Oean -------------------------------------------------------------------------------
        box=[-180, 180, -90, -50] 
        left, right, down, up = box
        mask_SO = ((mesh.x2 >= left) & 
                   (mesh.x2 <= right) & 
                   (mesh.y2 >= down) & 
                   (mesh.y2 <= up))

        #print(np.shape(mask_SO))
        
        # load nodal area -------------------------------------------------------------------------------------
        f = Dataset(self.resultpath + '/' + self.runname + '.mesh.diag.nc', 'r')
        nod_area = f.variables['nod_area'][:] # Moritz

        ## Southern Ocean
        nod_area_SO = nod_area[:,mask_SO]
        nod_area_SO_surface = nod_area[0,mask_SO]
        #print(np.shape(nod_area_SO_surface))
        
        # get depth of export production
        i_ep_depth = pf.ind_for_depth(100,mesh)
        nod_depth = f.variables['zbar'][:]
        ep_depth = nod_depth[i_ep_depth]
        #print('EP for selected depth = {0} m'.format(ep_depth))

        
        #print('shape nod_area: {0}\nshape nod_area_surface: {1}'.format(np.shape(nod_area),np.shape(nod_area_surface)))
        
        
        # calculating total NPP per year -------------------------------------------------------------------------------
        NPPtotal = np.zeros(len(years))
        DiaTotal  = np.zeros(len(years))
        NanoTotal = np.zeros(len(years))
        EPtotal   = np.zeros(len(years))
        SiEtotal   = np.zeros(len(years))
        
        for i in range(len(years)):
            ind_year = years[i]
            print('loading {0}'.format(ind_year))
            
            NPPn_global = pf.get_data(resultpath, "NPPn", ind_year, mesh, how="mean", compute=True, runid=self.runname, silent=True)
            NPPd_global = pf.get_data(resultpath, "NPPd", ind_year, mesh, how="mean", compute=True, runid=self.runname, silent=True)
            ## NPPn:units = "mmolC/(m2*d)"
            
            # apply Southern Ocean mask
            NPPn = NPPn_global[mask_SO]
            NPPd = NPPd_global[mask_SO]
            
            NPPtotal_aux       = (NPPd+NPPn)*12.01
            NPPtotal_aux       = 365 * NPPtotal_aux    # Conversion from [mg/m2/day]   => [mg/m2/yr]
            #print('shape NPPtotal: {0}'.format(np.shape(NPPtotal_aux)))
            
            NPPtotal[i] = np.sum(NPPtotal_aux*nod_area_SO_surface)/1e18  # summing up and converting to [Pg C/year]
        
            NanoTotal[i] = np.sum(12.01*365.*NPPn*nod_area_SO_surface)/1e18 
            DiaTotal[i]  = np.sum(12.01*365.*NPPd*nod_area_SO_surface)/1e18
            
            # Ozgur's results have different var_ID for DetC:
            #DetC = pf.get_data(resultpath, "tra_012", ind_year, mesh, how="mean", compute=True, runid=self.runname, silent=True)
            
            # org C export
            DetC_global = pf.get_data(resultpath, "DetC", ind_year, mesh, how="mean", compute=True, runid=self.runname, silent=True)
            DetC = DetC_global[mask_SO] ## Southern Ocean mask
            Vdet = 0.0288 * 100. + 20. ## sinking velocity
            DetC = 365. * DetC * 12.01 * Vdet # [mmol/m3] => [mg/m2/yr]
            detct=DetC[:,i_ep_depth]
            #EPtotal[i] = np.sum(detct*NodalArea2D[10,:])/1e18 # that's Ozgur's use of nodal area
            EPtotal[i] = np.nansum(detct * nod_area_SO[i_ep_depth,:])/1e18
            
            # Si export
            # DetSi:units = "[mmol/m3]"
            SiE_global = pf.get_data(resultpath, "DetSi", ind_year, mesh, how="mean", compute=True, runid=self.runname, silent=True)
            SiE = SiE_global[mask_SO] ## Souther Ocean mask
            SiE = 365. * SiE * Vdet # [mmol/m3] => [mmol/m2/yr]
            siect = SiE[:,i_ep_depth]
            SiEtotal[i] = np.nansum(siect * nod_area_SO[i_ep_depth,:])/1e15 # [Tmol Si/yr]
            
            
        
        # plotting total NPP -------------------------------------------------------------------------------        
        fig = plt.figure(figsize=(12,8), facecolor='w', edgecolor='k', tight_layout = True)
        plt.subplot(2, 3, 1)
        plt.plot(years,NPPtotal,'.-',label='Total')
        #plt.plot(years,NanoTotal,'.-g',label='Nano')
        #plt.plot(years,DiaTotal,'.-r',label='Dia')
        plt.title('Southern Ocean NPP')
        plt.ylabel(r'[Pg C yr$^{-1}$]')
        plt.legend(loc='upper right')
        
        plt.subplot(2, 3, 2)
        #plt.plot(years,NPPtotal,'.-',label='Total')
        plt.plot(years,NanoTotal,'.-g',label='Nano')
        #plt.plot(years,DiaTotal,'.-r',label='Dia')
        plt.title('SO NPP: nanophytoplankton')
        #plt.ylabel(r'[Pg C yr$^{-1}$]')
        plt.legend(loc='upper right')
        
        plt.subplot(2, 3, 3)
        #plt.plot(years,NPPtotal,'.-',label='Total')
        #plt.plot(years,NanoTotal,'.-g',label='Nano')
        plt.plot(years,DiaTotal,'.-r',label='Dia')
        plt.title('SO NPP: diatoms')
        #plt.ylabel(r'[Pg C yr$^{-1}$]')
        plt.legend(loc='upper right')
        
        # all NPP together
        plt.subplot(2, 3, 4)
        plt.plot(years,NPPtotal,'.-',label='Total')
        plt.plot(years,NanoTotal,'.-g',label='Nano')
        plt.plot(years,DiaTotal,'.-r',label='Dia')
        plt.title('Southern Ocean NPP')
        plt.ylabel(r'[Pg C yr$^{-1}$]')
        plt.legend(loc='upper right')

        # Export production
        plt.subplot(2, 3, 5)
        plt.plot(years,EPtotal,'.-')
        plt.title('Southern Ocean EP at {0} m'.format(ep_depth))
        plt.ylabel(r'[Pg C yr$^{-1}$]')
        
        # Si export
        plt.subplot(2, 3, 6)
        plt.plot(years,SiEtotal,'.-c')
        plt.title('Southern Ocean Si export at {0} m'.format(ep_depth))
        plt.ylabel(r'[Tmol Si yr$^{-1}$]')
        

        if(savefig):
            plt.savefig(self.savepath+self.runname+'_'+'NPP_Total_SouthernOcean_'+str(self.fyear)+'to'+str(self.lyear)+'.png', dpi = 300, bbox_inches='tight')
        plt.show(block=False)
        
    

In [ ]:
### TESTING ###
if __name__ == "__main__":
    import pyfesom2 as pf
    import numpy as np
    
    # run specification -------------------------------------------------------------------------------------
    runid      =  'fesom'
    resultpath = '/work/ollie/mozeisin/results/f2r1.2/' + 'mo12'
    first_year, last_year = 1948, 1980
    
    #runid = 'fesom'
    #resultpath ='/work/ollie/ogurses/output_f2r2/SECOND_RUN/'
    #first_year = 1949
    #last_year  = 1978

    savepath = '/home/ollie/mozeisin/evaluation/mo_files/'

    htmlname     =  runid + '_TEST_NPP_TotalGlobal.html'
    htmlpath = '/home/ollie/mozeisin/evaluation/mo_files/html/'

    meshpath = '/work/ollie/mozeisin/mesh/mesh_fesom2.0/core2_meanz'

    # WOA ------------------------------------------------------------------------------------    
#    ncpath = ncfileDIC
    #!ncdump -h $ncpath
    
    # now test:runname,resultpath,savepath,meshpath,first_year,last_year
    test = NPP_TotalSO(runid,resultpath,savepath,meshpath,first_year,last_year,layerwise=False,savefig=False)